In [18]:
import pandas as pd
import datetime
import time

import selenium # pip install selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from msedge.selenium_tools import EdgeOptions, Edge # pip install msedge-selenium-tools selenium==3.141

import re
import json
import lxml # pip install lxml
from lxml import etree
from bs4 import BeautifulSoup # pip install beautifulsoup4

from os import remove

def iniciar_driver():
    # El Web Driver debe estar en la misma ruta que el script
    options = EdgeOptions()
    options.use_chromium = True
    options.add_argument("headless")
    prefs = {"profile.managed_default_content_settings.images": 2}
    options.add_experimental_option("prefs", prefs)
    driver = Edge(options = options)
    return driver

def limite_posts():
    # Se ingresa la fecha hasta la que se guardan registros
    global fecha_limite
    while True:
        try:
            fecha_limite = input("Ingrese la fecha inicial en formato YYYY-MM-DD: ")
            fecha_limite = datetime.datetime.strptime(fecha_limite, '%Y-%m-%d')
            break
        except:
            print("El formato debe ser YYYY-MM-DD")
    fecha_limite = fecha_limite.strftime('%Y-%m-%d')
    fecha_limite = int(time.mktime(datetime.datetime.strptime(fecha_limite, '%Y-%m-%d').timetuple()))
    
def obtener_datos_posts(contenedor):
    """Extraer el timestamp y el link de los posts"""
    
    global control_fecha
    
    # timestamp
    timestamp = contenedor.find_element_by_xpath('.//span[@class="fsm fwn fcg"]/a/abbr'
                                ).get_attribute("data-utime")
    control_fecha = timestamp
    
    # Link
    try:
        enlace_post = contenedor.find_element_by_xpath('.//span[@class="text_exposed_link"]/a'
                                ).get_attribute("href")
    except NoSuchElementException:
        enlace_post = contenedor.find_element_by_xpath('.//span[@class="fsm fwn fcg"]/a'
                                ).get_attribute("href")

    time_link = (timestamp, enlace_post)
    return time_link

def collect_all_posts_from_current_view(driver, lookback_limit=19):
    contenedores = driver.find_elements_by_xpath('//div[@class="_4-u2 _4-u8"]')
    if len(contenedores) <= lookback_limit:
        return contenedores
    else:
        return contenedores[-lookback_limit:]

def obtener_datos(driver):
    """Obtener los links de los posts"""
    
    global datos_facebook
    post_ids = set()
    global control_fecha
    global fecha_limite
    # i = 0
    last_height = driver.execute_script("return window.pageYOffset;")
    scrolling = True

    while scrolling:
        # contenedores = driver.find_elements_by_xpath('//div[@class="_4-u2 _4-u8"]')
        contenedores = collect_all_posts_from_current_view(driver)
        for contenedor in contenedores:
            post = obtener_datos_posts(contenedor)
            # Verifíca la fecha límite
            if int(control_fecha) < fecha_limite:
                driver.close()
                return datos_facebook
            if post:
                post_id = ''.join(post)
                if post_id not in post_ids:
                    post_ids.add(post_id)
                    datos_facebook.append(post)
                    
        print(str(len(datos_facebook)) + " posts encontrados")
        
        intento_scroll = 0
        while True:
            # Scrollear navegador
            # i += 3000
            # driver.execute_script('window.scrollTo({ top: ' + str(i) + ', behavior: "smooth" });')
            # driver.execute_script('window.scrollTo({ top: document.body.scrollHeight, behavior: "smooth" });')
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Espera para cargar la página
            time.sleep(2)

            # Calcular la nueva posición del scroll y compararla con la anterior
            new_height = driver.execute_script("return window.pageYOffset;")
            if last_height == new_height:
                intento_scroll +=1
                
                # Fin de scroll
                if intento_scroll >= 3:
                    scrolling = False
                    break
                else:
                    time.sleep(2) # Intenta scrollear nuevamente
            else:
                last_height = new_height
                break

    driver.close()
    return datos_facebook

def obtener_info_post(script):
    global comentarios
    json_data = json.loads(script)
    
    # Id de la publicación
    id_post = json_data['identifier']
    try:
        id_post = re.search(';(.+?);;', id_post).group(1)
    except:
        id_post = re.search('(.+?):', id_post).group(1)
        
    # Fecha de publicación
    create_time = json_data['dateCreated']
    
    # Perfil
    profile = json_data['author']['name']
    
    # Texto de la publicación
    try:
        message = json_data['articleBody']
        message = message.replace('\n', ' ')
    except:
        message = ""
    
    # Enlace de la publicación
    link = json_data['url']
    
    # Número de reacciones
    reactions_count = json_data['interactionStatistic'][1]['userInteractionCount']
    
    # Número de compartidos
    shares_count = json_data['interactionStatistic'][2]['userInteractionCount']
    
    # Número de comentarios
    comments_count = json_data['commentCount']
    
    data_posts = (id_post, create_time, profile, message, link, reactions_count, shares_count, comments_count)
    
    
    # Información comentarios
    try:
        for i in json_data['comment']:
            try:
                id_comment = i['identifier']
                comment_time = i['dateCreated']
                user = i['author']['name']
                url_user = i['author']['url']
                if url_user == None:
                    url_user = "https://www.facebook.com/" + i['author']['identifier']
                comment = i['text'].replace('\n', '')
                url_comment = "https://www.facebook.com/" + i['identifier']
                original_post = message
            except:
                break
        
            data_comments = (id_comment, comment_time, user, url_user, comment, url_comment, original_post)
        
            comentarios.append(data_comments)
    except:
        pass
    
    return data_posts

def crear_dataframe(data, cols):
    data_frame = pd.DataFrame(data,
                      columns=cols,
                      index=None)
    return data_frame

url_facebook = "https://www.facebook.com/universidaddelsinumonteria/posts/"

driver = iniciar_driver()
driver.get(url_facebook)
time.sleep(2)

datos_facebook = []
control_fecha = ""
fecha_limite = ""

limite_posts()
print("Buscando posts...")
obtener_datos(driver)

print("Obteniendo información de posts...")
enlaces = []

for i in datos_facebook:
    enlaces.append(i[1])
    
data_facebook_posts = []

comentarios = []

driver = iniciar_driver()
    
for l in enlaces:
    driver.get(l)
    time.sleep(2)
    page = BeautifulSoup(driver.page_source, "html.parser").head
    
    try:
        script = page.find('script', attrs={"type": "application/ld+json"}).string
    except:
        continue

    script2 = page.find('script', attrs={"type": "application/ld+json"}).find_next_sibling('script').string
    
    try:
        fb_post = obtener_info_post(script)
    except:
        try:
            fb_post = obtener_info_post(script2)
        except:
            continue
    data_facebook_posts.append(fb_post)

driver.close()    
print("Completado")

Ingrese la fecha inicial en formato YYYY-MM-DD: 2021-01-01
Buscando posts...
19 posts encontrados
27 posts encontrados
35 posts encontrados
43 posts encontrados
51 posts encontrados
59 posts encontrados
67 posts encontrados
67 posts encontrados
83 posts encontrados
91 posts encontrados
99 posts encontrados
107 posts encontrados
115 posts encontrados
115 posts encontrados
131 posts encontrados
139 posts encontrados
147 posts encontrados
155 posts encontrados
163 posts encontrados
171 posts encontrados
179 posts encontrados
187 posts encontrados
187 posts encontrados
203 posts encontrados
203 posts encontrados
211 posts encontrados
227 posts encontrados
235 posts encontrados
235 posts encontrados
251 posts encontrados
259 posts encontrados
267 posts encontrados
275 posts encontrados
283 posts encontrados
291 posts encontrados
299 posts encontrados
307 posts encontrados
315 posts encontrados
315 posts encontrados
331 posts encontrados
339 posts encontrados
339 posts encontrados
347 posts 

In [19]:
columns_posts = ['id_post', 'create_time', 'profile', 'message', 'link', 'reactions', 'shares',
                'comments']

resultado = crear_dataframe(data_facebook_posts, columns_posts)
resultado

,id_post,create_time,profile,message,link,reactions,shares,comments
0,223570809885786,2021-11-22T09:18:40-0800,Universidad Del Sinú - Monteria,"El Ministro de Salud y Protección Social, doct...",https://www.facebook.com/universidaddelsinumon...,0,0,0
1,3158193657799741,2021-11-20T05:35:52-0800,Universidad Del Sinú - Monteria,¡Feliz día a todos los Psicólogos! Para los qu...,https://www.facebook.com/universidaddelsinumon...,18,0,0
2,1050969292353198,2021-11-19T11:37:20-0800,Universidad Del Sinú - Monteria,Conoce algunas experiencias de nuestros egresa...,https://www.facebook.com/universidaddelsinumon...,4,0,0
3,747530976206866,2021-11-19T11:35:56-0800,Universidad Del Sinú - Monteria,Conoce algunas experiencias de nuestros egresa...,https://www.facebook.com/universidaddelsinumon...,0,0,0
4,3157039681248472,2021-11-18T16:19:37-0800,Universidad Del Sinú - Monteria,Como reconocimiento a su loable desempeño en m...,https://www.facebook.com/universidaddelsinumon...,49,7,4
...,...,...,...,...,...,...,...,...
560,233674291700939,2021-01-27T13:34:43-0800,Universidad Del Sinú - Monteria,La Alta Calidad es una de las fortalezas del p...,https://www.facebook.com/universidaddelsinumon...,23,3,0
561,133322398626419,2021-01-27T06:20:49-0800,Universidad Del Sinú - Monteria,Compartimos con ustedes los logros de nuestro ...,https://www.facebook.com/universidaddelsinumon...,21,0,0
562,1121132761691942,2021-01-26T12:27:24-0800,Universidad Del Sinú - Monteria,La Universidad del Sinú- Elías Bechara Zainúm ...,https://www.facebook.com/universidaddelsinumon...,66,11,0
563,2925883391030770,2021-01-25T03:40:43-0800,Universidad Del Sinú - Monteria,¡Bienvenidos! Unisinú te recibe y está dispues...,https://www.facebook.com/universidaddelsinumon...,163,13,0


In [23]:
resultado.to_excel("postsFacebook.xlsx")

In [24]:
columns_comments = ['id_comment', 'create_time', 'user', 'url_user', 'comment', 'url_comment', 'original_post']

resultado_comments = crear_dataframe(comentarios, columns_comments)

resultado_comments

,id_comment,create_time,user,url_user,comment,url_comment,original_post
0,3157039681248472_3157053634580410,2021-11-18T16:50:21-0800,Robinson Bayona,https://www.facebook.com/people/Robinson-Bayon...,Con mas razón deben comenzar clases presenciales,https://www.facebook.com/3157039681248472_3157...,Como reconocimiento a su loable desempeño en m...
1,3157039681248472_3157053644580409,2021-11-18T16:50:22-0800,Cenilda Martinez,https://www.facebook.com/cenilda.martinez.3,El ministro no le importa la salud por qu...,https://www.facebook.com/3157039681248472_3157...,Como reconocimiento a su loable desempeño en m...
2,3157039681248472_3157380271214413,2021-11-19T03:53:25-0800,Campo Elias Vergara Pretelt,https://www.facebook.com/1559017152,Así como se hacen actos protocolarios y concie...,https://www.facebook.com/3157039681248472_3157...,Como reconocimiento a su loable desempeño en m...
3,3157039681248472_3157706951181745,2021-11-19T13:39:12-0800,Bob Talam,https://www.facebook.com/100000134627509,🎬. ESTAS SON LAS VACUNAS QUE MUCHOS MÉDICOSLE ...,https://www.facebook.com/3157039681248472_3157...,Como reconocimiento a su loable desempeño en m...
4,3154920918127015_3155064548112652,2021-11-16T08:01:37-0800,Kevin-kun Gaviria,https://www.facebook.com/people/Kevin-kun-Gavi...,Gracias a Andrea ortega y la universidad del s...,https://www.facebook.com/3154920918127015_3155...,No te pierdas el video de cierre de la Campaña...
...,...,...,...,...,...,...,...
1414,400934504348808_401003207675271,2021-01-15T05:39:41-0800,Gregoria Arteaga Morales,https://www.facebook.com/gregoria.arteaga.79,Amen 🙌,https://www.facebook.com/400934504348808_40100...,Misa - Conmemoración vigésimo aniversario del ...
1415,400934504348808_402458160863109,2021-01-18T04:15:17-0800,Luis Perez Gonzalez,https://www.facebook.com/luchoperezgonzalez,QEPD,https://www.facebook.com/400934504348808_40245...,Misa - Conmemoración vigésimo aniversario del ...
1416,400934504348808_401315780977347,2021-01-15T19:40:24-0800,Guillermo Aranguren Villate,https://www.facebook.com/guillermo.arangurenvi...,🙏,https://www.facebook.com/400934504348808_40131...,Misa - Conmemoración vigésimo aniversario del ...
1417,400934504348808_400984017677190,2021-01-15T05:11:36-0800,Zonia Bechara De Barguil,https://www.facebook.com/559894324,❤️❤️🙏🙏,https://www.facebook.com/400934504348808_40098...,Misa - Conmemoración vigésimo aniversario del ...


In [25]:
resultado_comments.to_excel("commentsFacebook.xlsx")